In [4]:
import h2o
from h2o.automl import H2OAutoML
import subprocess
subprocess.run('unset http_proxy', shell=True)
h2o.init()


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,41 mins 38 secs
H2O cluster timezone:,UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.1
H2O cluster version age:,13 days
H2O cluster name:,H2O_from_python_siemanalyst_s98360
H2O cluster total nodes:,1
H2O cluster free memory:,4.699 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"locked, healthy"


In [7]:
import pandas as pd
import numpy as np
from pandas import DataFrame

#from typing import Set, Any
#def remove_others(df: DataFrame, columns: Set[Any]):
#    cols_total: Set[Any] = set(df.columns)
#    diff: Set[Any] = cols_total - columns
#    df.drop(diff, axis=1, inplace=True)
    
### From Phishtank Import Malware urls

phishtank_trainpd=pd.read_csv("/home/siemanalyst/notebooks/siem/online-valid.csv")
phishtank_trainds=phishtank_trainpd.drop(columns=['phish_id','phish_detail_url','submission_time','verification_time','online','target']).head(400)
phishtank_trainds['url_length'] = phishtank_trainds['url'].map(str).apply(len)
phishtank_trainds['massiveurl'] = np.where(phishtank_trainds['url_length'] > 100 , 1, 0)
phishtank_trainds.head(6)
train=h2o.H2OFrame(phishtank_trainds)

### From Commom Crawl Import global urls :: Asume as verified=0
ccrawl_trainpd=pd.read_json('/home/siemanalyst/notebooks/siem/tail-CC-MAIN-2018-43-index-.com.json', typ='series').to_frame('count')
ccrawl_trainpd.head(100)
#ccrawl_trainds=remove_others(ccrawl_trainpd, {"url"})
#ccrawl_trainds.head(3)

phishtank_testpd=pd.read_csv("/home/siemanalyst/notebooks/siem/online-valid.csv")
phishtank_testds=phishtank_testpd.drop(columns=['phish_id','phish_detail_url','submission_time','verification_time','online','target']).tail(10)
phishtank_testds['url_length'] = phishtank_testds['url'].map(str).apply(len)
phishtank_testds['massiveurl'] = np.where(phishtank_testds['url_length'] > 100 , 1, 0)
phishtank_testds.head(6)
test=h2o.H2OFrame(phishtank_testds)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [10]:
# Import a sample binary outcome train/test set into H2O
#train = h2o.import_file("/home/oracle/notebooks/seim/online-valid.csv")
#test = h2o.import_file("/home/oracle/notebooks/seim/online-valid.csv").head(2000)

train.head()
train['massiveurl'].describe()
test.head()
test['massiveurl'].describe()


Rows:400
Cols:1




,massiveurl
type,int
mins,0.0
mean,0.1475
maxs,1.0
sigma,0.35504773938599826
zeros,341
missing,0
0,1.0
1,0.0
2,0.0


Rows:10
Cols:1




,massiveurl
type,int
mins,0.0
mean,0.0
maxs,0.0
sigma,0.0
zeros,10
missing,0
0,0.0
1,0.0
2,0.0


In [11]:
# Identify predictors and response
x = train.columns
y = "massiveurl"
x.remove(y)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20, seed=1999)
aml.train(x=x, y=y, training_frame=train)


AutoML progress: |████████████████████████████████████████████████████████| 100%


In [12]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,auc,logloss,mean_per_class_error,rmse,mse
XGBoost_1_AutoML_20181109_165017,1,0.0255565,0,0.0384084,0.00147521
XGBoost_grid_1_AutoML_20181109_165017_model_4,1,0.00884463,0,0.0124791,0.000155728
StackedEnsemble_AllModels_AutoML_20181109_165017,1,5.39981e-05,0,9.20297e-05,8.46946e-09
XGBoost_grid_1_AutoML_20181109_165017_model_2,1,0.00655446,0,0.00913771,8.34977e-05
GBM_1_AutoML_20181109_165017,1,0,0,0,0
GLM_grid_1_AutoML_20181109_165017_model_1,1,0.0166185,0,0.0595585,0.00354721
XRT_1_AutoML_20181109_165017,1,0.00349934,0,0.0377512,0.00142515
XGBoost_grid_1_AutoML_20181109_165017_model_1,1,0.0206397,0,0.0281793,0.000794071
DRF_1_AutoML_20181109_165017,1,0,0,0,0
GBM_3_AutoML_20181109_165017,1,0,0,0,0
